깃허브 코드와 데이터를 클론해오기

In [1]:
!git clone https://github.com/Gwakcy0/IssueHorizon.git

Hello World!


사용해야 하는 모듈 설치하기

In [ ]:
%cd IssueHorizon
# !pip install -r "requirements.txt"

* 참고 링크: https://github.com/KPFBERT/kpfbertsum/blob/main/kpfbert_summary.ipynb
* 사용 데이터: ETRI의 AI-HUB에서 제공하는 문서요약 텍스트(비플라이소프트) 
-> https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=97

전체기사에서 중요한 순서대로 상위 3개의 문장을 추출해서 제시하는 기사 요약 서비스이다.

pytorch-lightning을 이용하여 전체프로세서를 작성하였다.

BERT를 이용한 SUMMARY 관련 논문 및 nlpyang의 PreSumm 소스를 참조하였다.

In [1]:
import math
import pandas as pd
import numpy as np

from tqdm.auto import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import torch.optim as optim

from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup

from torch.nn.init import xavier_uniform_

import pytorch_lightning as pl
# from pytorch_lightning.metrics.functional import accuracy, f1, auroc
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

import os

# import kss

# %matplotlib inline
# %config InlineBackend.figure_format='retina'

RANDOM_SEED = 42

sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8

print(pl.seed_everything(RANDOM_SEED))

MAX_TOKEN_COUNT = 512
N_EPOCHS = 10
BATCH_SIZE = 4

c:\Users\user\anaconda3\envs\horizon\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Global seed set to 42


42


In [2]:
'''
Data는 AI-HUB의 문서요약 텍스트의 json 파일을 사용한다.
'''

cwd = os.getcwd()
DATA_TRAIN_PATH = os.path.join(cwd, 'data', 'train_original_0.json')
# DATA_TRAIN_PATH = 'data/train_original_0.json'
df = pd.read_json(DATA_TRAIN_PATH)
df = df.dropna()
print('training data len: ', len(df))

DATA_TEST_PATH = os.path.join(cwd, 'data', 'valid_original_0.json')
# DATA_TEST_PATH = 'data/vaild_original_0.json'
test_df = pd.read_json(DATA_TEST_PATH)
test_df = test_df.dropna()
print('test data len: ', len(test_df))

# 데이터셋이 두개인 관계로 training set을 한 번 더 쪼개서 validation set을 만든다.
train_df, val_df = train_test_split(df, test_size=0.05)
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
print('train/validation/test shape: ', train_df.shape, val_df.shape, test_df.shape)

ValueError: Expected object or value

In [ ]:
# test setting으로 모든 데이터 다운사이즈
downsize = 2000
train_df = train_df[:downsize]
test_df = test_df[:downsize//10]
val_df = val_df[:downsize//10]
print('train/validation/test downsized: ', train_df.shape, val_df.shape, test_df.shape)

In [ ]:
# json 파일 혹은 딕셔너리 데이터를 dataframe으로 바꾸기
def preprocess_data(data):
    outs = []
    for doc in data['documents']:
        line = []
        line.append(doc.get('media_name'))
        line.append(doc['id'])
        para = []
        for sent in doc['text']:
            for s in sent:
                para.append(s['sentence'])
        line.append(para)
        line.append(doc['abstractive'][0])
        line.append(doc['extractive'])
        a = doc['extractive']
        if a[0] == None or a[1] == None or a[2] == None:
            continue
        outs.append(line)

    outs_df = pd.DataFrame(outs)
    outs_df.columns = ['media', 'id', 'article_original', 'abstractive', 'extractive']
    return outs_df


In [ ]:
print(train_df.head(1))
train_df = preprocess_data(train_df)
print(train_df.head(1))

In [ ]:
# 본문 프린트해보기
i = 8
print('===== 본    문 =====')
for idx, str in enumerate(train_df['article_original'][i]):
    print(idx,':',str)
print('===== 요약정답 =====')
print(train_df['extractive'][i])
print('===== 추출본문 =====')
print('1 :', train_df['article_original'][i][train_df['extractive'][i][0]])
print('2 :', train_df['article_original'][i][train_df['extractive'][i][1]])
print('3 :', train_df['article_original'][i][train_df['extractive'][i][2]])
print('===== 생성본문 =====')
print(train_df['abstractive'][i])
